## Team Stat Calculations
Author: Akshay Easwaran <akeaswaran@me.com>

---
Based on work by ESPN's Bill Connelly, Football Outsiders, and Football Study Hall

Data from CollegeFootballData.com

Useful Resources for Stat Definitions:
- Football Study Hall website: https://www.footballstudyhall.com/
- Football Study Hall book: https://www.amazon.com/Study-Hall-College-Football-Stories/dp/1484989961
- Football Outsiders: https://www.footballoutsiders.com/info/glossary
- Example Advanced Box Score: https://twitter.com/ESPN_BillC/status/1176572006969597954/photo/1

**How to use this Notebook**

1. Dump a team's play by play data, game data, and drive data from CollegeFootballData.com.
2. Change the file names in the next cell to match where your data files are located.
3. Change the away and home team names appropriately.
4. Hit Cell > Run All Cells.

In [21]:
import requests
import pandas as pd
import json
import html
import os.path

selected_team = 'Virginia Tech'

def retrieveCfbData(endpoint, team):
    file_path = f"data/{endpoint if (endpoint != 'plays') else 'pbp'}/{endpoint[:-1] if (endpoint != 'plays') else 'pbp'}-data-{team.lower().replace(' ','-')}.json"
    if (os.path.exists(file_path)):
        return file_path
    res = requests.get(f"https://api.collegefootballdata.com/{endpoint}?seasonType=regular&year=2019&team={html.escape(team)}")
    content = res.json()
    with open(file_path, 'w') as f:
        json.dump(content, f)
    return json.dumps(content)
    

pbp_data = pd.read_json(retrieveCfbData('plays',selected_team))
# pbp_data.info()

base_drives = pd.read_json(retrieveCfbData('drives',selected_team))
base_drives.drop(['offense_conference','start_time','end_time','defense_conference','elapsed','start_period','end_period'], axis = 1, inplace=True) 
base_drives = base_drives[
    ~base_drives.drive_result.isin(['END OF HALF','END OF GAME','Uncategorized'])
]

games = pd.read_json(retrieveCfbData('games',selected_team))

In [22]:
# Data Cleaning
# Fix the bad yard line markers for away teams
drives = pd.merge(base_drives, games[['id','away_team','home_team']], left_on='game_id', right_on='id', how='right')
drives.rename(columns={'id_x':'drive_id'}, inplace=True)
drives.drop(['id_y'], axis = 1, inplace=True)
drives.dropna(inplace=True)
drives.loc[
    drives.offense == drives.away_team, ['start_yardline']
] = 100 - drives.start_yardline
drives.loc[
    drives.offense == drives.away_team, ['end_yardline']
] = 100 - drives.end_yardline

print("Total Drives:", len(drives))
# drives

Total Drives: 220


In [23]:
# Eliminate garbage time plays
garbage_time_data = pbp_data[
    (pbp_data.down == 0)
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 38) & (pbp_data.period == 2))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 28) & (pbp_data.period == 3))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 22) & (pbp_data.period == 4))
]

print("Total Plays:", len(pbp_data))
print("Garbage Time Plays:", len(garbage_time_data))
pbp_data.drop(garbage_time_data.index, inplace=True)
print("Non-Garbage Time Plays:", len(pbp_data))

# Drop some unnecessary columns
pbp_data.drop(['offense_conference','defense_conference','clock'], axis = 1, inplace=True) 
# Ignore some types of plays cause they're special teams and weird
ignore_types = ["Defensive 2pt Conversion","Blocked Field Goal","Blocked Punt","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]
pbp_data = pbp_data[~(pbp_data.play_type.isin(ignore_types))]
print("Total clean plays:", len(pbp_data))

Total Plays: 1701
Garbage Time Plays: 71
Non-Garbage Time Plays: 1630
Total clean plays: 1264


In [24]:
import numpy as np

pbp_data.distance = pbp_data.distance.astype(float)

bad_types = ["Interception","Pass Interception Return","Interception Return Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

def is_successful(down, distance, yards_gained, play_type):
    if (play_type in bad_types):
        return False 
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif (((down == 3) | (down == 4)) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp_data['play_successful'] = np.vectorize(is_successful)(pbp_data.down, pbp_data.distance, pbp_data.yards_gained, pbp_data.play_type)
pbp_data.play_successful.value_counts()

def is_explosive(yards_gained):
    if (yards_gained >= 15):
        return True
    else:
        return False
    
pbp_data['play_explosive'] = np.vectorize(is_explosive)(pbp_data.yards_gained)

In [25]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown","Interception","Pass Interception Return","Interception Return Touchdown"]
rush_types = ["Rush","Rushing Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

print("Offensive Success Rates")
print(selected_team)
print("Overall:",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == selected_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_type.isin(rush_types))]))
print("")
print("Success Rates Allowed")
print(selected_team)
print("Overall:",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.defense == selected_team)]))
print("Passing:",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_type.isin(rush_types))]))

Offensive Success Rates
Virginia Tech
Overall: 0.4051863857374392
Passing: 0.46320346320346323
Rushing: 0.3733681462140992

Success Rates Allowed
Virginia Tech
Overall: 0.3755795981452859
Passing: 0.4046920821114369
Rushing: 0.3465346534653465


In [26]:
downs = [1, 2, 3, 4]
print("Success Rates on Specific Downs (Off/Allowed)")
print(selected_team)
for d in downs:
    intermed = pbp_data[
        (pbp_data.down == d)
    ]
    print(f"Down {d}: {len(intermed[(intermed.play_successful == True) & (intermed.offense == selected_team)]) / len(intermed[(intermed.offense == selected_team)])} / {len(intermed[(intermed.play_successful == True) & (intermed.defense == selected_team)]) / len(intermed[(intermed.defense == selected_team)])}")

Success Rates on Specific Downs (Off/Allowed)
Virginia Tech
Down 1: 0.3890909090909091 / 0.34814814814814815
Down 2: 0.39901477832512317 / 0.3691588785046729
Down 3: 0.4230769230769231 / 0.4068965517241379
Down 4: 0.7777777777777778 / 0.6111111111111112


In [27]:
# Explosiveness rates
# Defined as rate of 15+ yards gains

print("Explosiveness Plays (Rates)")
print(selected_team)
print("Overall:",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_explosive == True)]),"/", len(pbp_data[(pbp_data.offense == selected_team)]),"(",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == selected_team)]),")")
print("Passing:",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]),"/", len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_type.isin(pass_types))]),"(",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_type.isin(pass_types))]),")")
print("Rushing:",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]),"/", len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_type.isin(rush_types))]),"(",len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == selected_team) & (pbp_data.play_type.isin(rush_types))]),")")
print("")
print("Explosiveness Plays Allowed (Rates)")
print(selected_team)
print("Overall:",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_explosive == True)]),"/", len(pbp_data[(pbp_data.defense == selected_team)]),"(",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.defense == selected_team)]),")")
print("Passing:",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]),"/", len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_type.isin(pass_types))]),"(",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_type.isin(pass_types))]),")")
print("Rushing:",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]),"/", len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_type.isin(rush_types))]),"(",len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.defense == selected_team) & (pbp_data.play_type.isin(rush_types))]),")")

Explosiveness Plays (Rates)
Virginia Tech
Overall: 72 / 617 ( 0.1166936790923825 )
Passing: 44 / 231 ( 0.19047619047619047 )
Rushing: 28 / 383 ( 0.0731070496083551 )

Explosiveness Plays Allowed (Rates)
Virginia Tech
Overall: 70 / 647 ( 0.10819165378670788 )
Passing: 50 / 341 ( 0.1466275659824047 )
Rushing: 20 / 303 ( 0.066006600660066 )


In [28]:
# Havoc plays
havoc_plays = pbp_data[
    (((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type == 'Fumble Recovery (Opponent)')
    | (pbp_data.play_type == 'Sack')
    | (pbp_data.play_type.str.contains('Interception', regex=False))
    | (pbp_data.yards_gained < 0))
    & (pbp_data.play_type != 'Penalty')
]

print(selected_team)
print("Havoc Caused Rate: ", len(havoc_plays[
    havoc_plays.defense == selected_team
]), "/", len(pbp_data[
    pbp_data.defense == selected_team
]), "(",len(havoc_plays[
    havoc_plays.defense == selected_team
]) / len(pbp_data[
    pbp_data.defense == selected_team
]),")")
print("Havoc Suffered Rate: ", len(havoc_plays[
    havoc_plays.offense == selected_team
]), "/", len(pbp_data[
    pbp_data.offense == selected_team
]), "(",len(havoc_plays[
    havoc_plays.offense == selected_team
]) / len(pbp_data[
    pbp_data.offense == selected_team
]),")")

Virginia Tech
Havoc Caused Rate:  106 / 647 ( 0.16383307573415765 )
Havoc Suffered Rate:  91 / 617 ( 0.14748784440842788 )


In [29]:
# Calculating converted 3rd and 8+s
ignore_turnovers = ['Fumble Recovery (Opponent)','Fumble Return Touchdown',"Interception","Pass Interception Return","Interception Return Touchdown"]

third_longs = pbp_data[
    (pbp_data.down == 3)
    & (pbp_data.distance >= 8)
    & (~pbp_data.play_type.isin(ignore_turnovers))
]
# converted_3rd_longs

selected_team_thirds = third_longs[
    (third_longs.offense == selected_team)
]

selected_team_thirds_converted = selected_team_thirds[
    (selected_team_thirds.yards_gained >= 8)
]
print("Converted 3rd and 8+ for", selected_team,":",len(selected_team_thirds_converted),"of",len(selected_team_thirds))

home_team_thirds = third_longs[
    (third_longs.defense == selected_team)
]

home_team_thirds_converted = home_team_thirds[
    (home_team_thirds.yards_gained >= 8)
]

print("Converted 3rd and 8+ allowed by", selected_team,":",len(home_team_thirds_converted),"of",len(home_team_thirds))

Converted 3rd and 8+ for Virginia Tech : 21 of 50
Converted 3rd and 8+ allowed by Virginia Tech : 24 of 58


In [30]:
selected_team_drives = drives[
    drives.offense == selected_team
]
print(selected_team)
print("Drives:", len(selected_team_drives))
print("Yards:",sum(selected_team_drives.yards))
print("Plays:",sum(selected_team_drives.plays))
print("Avg Starting Field Position:", selected_team_drives.start_yardline.mean())
print("Yards per Play:", sum(selected_team_drives.yards) / sum(selected_team_drives.plays))
print("Plays per Drive:", sum(selected_team_drives.plays) / len(selected_team_drives))
print("Yards per Drive:", sum(selected_team_drives.yards) / len(selected_team_drives))

print("")

opp_team_drives = drives[
    drives.defense == selected_team
]

print("Opponents")
print("Drives:", len(opp_team_drives))
print("Yards:",sum(opp_team_drives.yards))
print("Plays:",sum(opp_team_drives.plays))
print("Avg Starting Field Position:", opp_team_drives.start_yardline.mean())
print("Yards per Play:", sum(opp_team_drives.yards) / sum(opp_team_drives.plays))
print("Plays per Drive:", sum(opp_team_drives.plays) / len(opp_team_drives))
print("Yards per Drive:", sum(opp_team_drives.yards) / len(opp_team_drives))

Virginia Tech
Drives: 110
Yards: 3507.0
Plays: 623.0
Avg Starting Field Position: 30.081818181818182
Yards per Play: 5.629213483146067
Plays per Drive: 5.663636363636364
Yards per Drive: 31.881818181818183

Opponents
Drives: 110
Yards: 3390.0
Plays: 642.0
Avg Starting Field Position: 30.754545454545454
Yards per Play: 5.280373831775701
Plays per Drive: 5.836363636363636
Yards per Drive: 30.818181818181817


In [31]:
# Measuring success rate for a single player
# pbp_data[
#     (pbp_data.play_text.str.contains("Quentin Harris"))
#     & (pbp_data.play_type.isin(pass_types))
#     & (~pbp_data.play_type.str.contains("Sack"))
# ].play_successful.value_counts(normalize=True)

In [32]:
# Standard vs Passing Downs success rates
# Success rate on standard downs == leverage rate

standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]

pass_plays = pbp_data[
    pbp_data.play_type.isin(pass_types)
]
rush_plays = pbp_data[
    pbp_data.play_type.isin(rush_types)
]

In [33]:
print(selected_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == selected_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == selected_team)
]))
print("Opp Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.defense == selected_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.defense == selected_team)
]))

print(selected_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == selected_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == selected_team)
]))

print("Opp Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.defense == selected_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.defense == selected_team)
]))

Virginia Tech Success Rate on Std Downs: 0.43675417661097854
Opp Success Rate on Std Downs: 0.3824228028503563
Virginia Tech Success Rate on Passing Downs: 0.3383838383838384
Opp Success Rate on Passing Downs: 0.36283185840707965


In [34]:
# Stuff Rate
stuffed_plays = rush_plays[
    (rush_plays.yards_gained <= 0)
    & (rush_plays.play_type != 'Sack')
]

In [35]:
selected_team_stuffs = stuffed_plays[
    stuffed_plays.defense == selected_team
]
print(selected_team,"Defensive Stuff Rate: ",len(selected_team_stuffs),"/",len(rush_plays[rush_plays.defense == selected_team]),"(",len(selected_team_stuffs)/len(rush_plays[rush_plays.defense == selected_team]),")")
print(selected_team,"Defensive Stuffs Suffered Rate: ",len(stuffed_plays[
    stuffed_plays.offense == selected_team
]),"/",len(rush_plays[rush_plays.offense == selected_team]),"(",len(stuffed_plays[
    stuffed_plays.offense == selected_team
])/len(rush_plays[rush_plays.offense == selected_team]),")")


Virginia Tech Defensive Stuff Rate:  59 / 303 ( 0.19471947194719472 )
Virginia Tech Defensive Stuffs Suffered Rate:  68 / 383 ( 0.17754569190600522 )


In [36]:
# Opportunity Rate
rush_opps = rush_plays[
    (rush_plays.yards_gained >= 5)
]
print(selected_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == selected_team]),"/",len(rush_plays[rush_plays.offense == selected_team]),"(",len(rush_opps[rush_opps.offense == selected_team])/len(rush_plays[rush_plays.offense == selected_team]),")")
print(selected_team,"Rush Opp Allowed Rate: ",len(rush_opps[rush_opps.defense == selected_team]),"/",len(rush_plays[rush_plays.defense == selected_team]),"(",len(rush_opps[rush_opps.defense == selected_team])/len(rush_plays[rush_plays.defense == selected_team]),")")

Virginia Tech Rush Opp Rate:  128 / 383 ( 0.3342036553524804 )
Virginia Tech Rush Opp Allowed Rate:  99 / 303 ( 0.32673267326732675 )


In [37]:
# Line Yards -- not adjusted for down/distance/opponent/shotgun
def adjust_strength_for_ol(yards_gained):
    if (yards_gained < 0):
        return yards_gained * 1.25
    elif ((yards_gained >= 0) & (yards_gained <= 3)):
        return yards_gained * 1.0
    elif ((yards_gained >= 4) & (yards_gained <= 6)):
        return yards_gained * 0.5
    else:
        return 0

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - adjust_strength_for_ol(x.yards_gained), axis=1)
print(selected_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == selected_team].line_yards) / len(rush_plays[rush_plays.offense == selected_team]))
print(selected_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == selected_team].highlight_yards) / len(rush_plays[rush_plays.offense == selected_team]))
print(selected_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == selected_team].yards_gained) / len(rush_plays[rush_plays.offense == selected_team]))

print("")

print("Opp Line Yards per Carry:",sum(rush_plays[rush_plays.defense == selected_team].line_yards) / len(rush_plays[rush_plays.defense == selected_team]))
print("Opp Highlight Yards per Carry:",sum(rush_plays[rush_plays.defense == selected_team].highlight_yards) / len(rush_plays[rush_plays.defense == selected_team]))
print("Opp Yards per Carry:",sum(rush_plays[rush_plays.defense == selected_team].yards_gained) / len(rush_plays[rush_plays.defense == selected_team]))

Virginia Tech Line Yards per Carry: 0.34464751958224543
Virginia Tech Highlight Yards per Carry: 3.9582245430809397
Virginia Tech Yards per Carry: 4.302872062663186

Opp Line Yards per Carry: 0.08498349834983498
Opp Highlight Yards per Carry: 3.957920792079208
Opp Yards per Carry: 4.042904290429043


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [38]:
# Expected and Adjusted Turnover Margins

adj_turnover_plays = pbp_data[
    (pbp_data.play_type.str.contains('Interception', regex=False))
    | ((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type.str.contains('Fumble', regex=False))
]

adj_fum = 0.5 * len(adj_turnover_plays[
    (adj_turnover_plays.play_type.str.contains('Fumble', regex=False))
])

# selected_team Adj Turnovers
selected_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == selected_team)
    | (adj_turnover_plays.defense == selected_team)
]
print(selected_team)
selected_team_ints_def = len(selected_team_tos[
   (selected_team_tos.play_type.str.contains('Interception', regex=False))
    & (selected_team_tos.defense == selected_team)
])

selected_team_ints_off = len(selected_team_tos[
   (selected_team_tos.play_type.str.contains('Interception', regex=False))
    & (selected_team_tos.offense == selected_team)
])

selected_team_pds = len(selected_team_tos[
    (((selected_team_tos.play_type == 'Pass Incompletion') & (selected_team_tos.play_text.str.contains('broken up', regex=False))) 
     | (selected_team_tos.play_type.str.contains('Interception', regex=False)))
    & (selected_team_tos.offense == selected_team)
])

selected_team_fum_rec = selected_team_tos[(selected_team_tos.play_type == 'Fumble Recovery (Opponent)') & (selected_team_tos.defense == selected_team)]
selected_team_fum_lost = selected_team_tos[(selected_team_tos.play_type == 'Fumble Recovery (Opponent)') & (selected_team_tos.offense == selected_team)]

print("Def INTs:",selected_team_ints_def)
print("Off INTs:",selected_team_ints_off)
print("PDs:",selected_team_pds)
print("Exp INTs:",0.22 * selected_team_pds)
print("Fum Recovered:",len(selected_team_fum_rec))
print("Fum Lost:",len(selected_team_fum_lost))
print("Exp Fum:",adj_fum)
print("Actual TO:",selected_team_ints_off + len(selected_team_fum_lost))
print("Actual TO Margin:",(selected_team_ints_def + len(selected_team_fum_rec) - selected_team_ints_off) - len(selected_team_fum_lost))
selected_team_exp_to = (0.22 * selected_team_pds) + adj_fum
print("Exp TO:",selected_team_exp_to)

Virginia Tech
Def INTs: 10
Off INTs: 5
PDs: 22
Exp INTs: 4.84
Fum Recovered: 2
Fum Lost: 9
Exp Fum: 9.0
Actual TO: 14
Actual TO Margin: -2
Exp TO: 13.84


In [39]:
to_luck = (selected_team_ints_off + len(selected_team_fum_lost) - selected_team_exp_to)
print("TO Luck for",selected_team,":",to_luck*5.0)
print("TO Luck/gm for",selected_team,":",(to_luck*5.0 / len(games.notna())))

TO Luck for Virginia Tech : 0.8000000000000007
TO Luck/gm for Virginia Tech : 0.06666666666666672


In [40]:
# Scoring Opportunities
# Definition: roughly, any time you get inside the opponent's 40, you should probably score

scoring_opps = drives[
    ((drives.start_yardline + drives.yards) >= 60)
]

# away team's scoring opps
print("Scoring Opportunities (IE: Drives inside Opponent's 40)")
print(selected_team)
selected_team_scoring_opps = scoring_opps[
    scoring_opps.offense == selected_team
]
print("Total:",len(selected_team_scoring_opps))
print("Scored:",len(selected_team_scoring_opps[selected_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(selected_team_scoring_opps[selected_team_scoring_opps.scoring == True]) / len(selected_team_scoring_opps))
print("Opps/Drive:",len(selected_team_scoring_opps) / len(drives[drives.offense == selected_team]))
print("Points/Opp:",(len(selected_team_scoring_opps[selected_team_scoring_opps.drive_result == 'TD']) * 7 + len(selected_team_scoring_opps[selected_team_scoring_opps.drive_result == 'FG'] * 3)) / len(selected_team_scoring_opps))

print("")
# Opponents' scoring opps
print("Opponents")
def_scoring_opps = scoring_opps[
    scoring_opps.defense == selected_team
]
print("Total:",len(def_scoring_opps))
print("Scored:",len(def_scoring_opps[def_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(def_scoring_opps[def_scoring_opps.scoring == True]) / len(def_scoring_opps))
print("Opps/Drive:",len(def_scoring_opps) / len(drives[drives.defense == selected_team]))
print("Points/Opp:",(len(def_scoring_opps[def_scoring_opps.drive_result == 'TD']) * 7 + len(def_scoring_opps[def_scoring_opps.drive_result == 'FG'] * 3)) / len(def_scoring_opps))

Scoring Opportunities (IE: Drives inside Opponent's 40)
Virginia Tech
Total: 54
Scored: 44
Opp Efficiency: 0.8148148148148148
Opps/Drive: 0.4909090909090909
Points/Opp: 4.37037037037037

Opponents
Total: 56
Scored: 40
Opp Efficiency: 0.7142857142857143
Opps/Drive: 0.509090909090909
Points/Opp: 3.9107142857142856


# 